In [1]:
%load_ext autoreload

In [20]:
import os
import pandas as pd
from common import sessions, sessfile, load_metadata
root = '/Users/lukearend/phd/kfx/data/mnt/Ketamine'

In [21]:
from tqdm import tqdm

metadata = {}
missing = '/Users/lukearend/phd/kfx/ref/2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.meta'
for region in ['hpc', 'pfc']:
    metadata[region] = {}
    for signal in ['lfp', 'ap']:
        df = []
        for i, sess in tqdm(sessions.iterrows(), total=36):
            metafile = sessfile(sess.path, region, signal, 'meta')
            path = os.path.join(root, metafile)
            try:
                md = pd.Series(load_metadata(path))
            except FileNotFoundError:
                md = pd.Series(load_metadata(missing))
            df.append(md)
        metadata[region][signal] = pd.DataFrame(df, index=sessions.index)

100%|███████████████████████████████████████████| 36/36 [00:01<00:00, 23.80it/s]


In [39]:
out = {}
for region in ['pfc', 'hpc']:
    out[region] = {}
    for signal in ['lfp', 'ap']:
        md = metadata[region][signal].loc[0]
        out[region][signal] = {
            'gains': md.imroTbl,
            'chans': md.snsChanMap,
            'shanks': md.snsShankMap if signal == 'ap' else None,
        }

In [49]:
for region in ['pfc', 'hpc']:
    for signal in ['ap', 'lfp']:
        gains = out['pfc']['lfp']['gains']
        gains = gains.strip('()').split(')(')
        gains = gains[1:]
        gains = [s.split(' ')[1:] for s in gains]
        for g in gains:
            assert g == gains[0]

In [32]:
df1 = sessions.drop(columns='path')
df2 = df.drop(columns=params.index)
df1.merge(df2, left_index=True, right_index=True).head()

,indiv,date,drug,doses,fileCreateTime,fileSizeBytes,fileTimeSecs,firstSample
recording,,,,,,,,
0,M015,2022-07-27,SAL,"(0, 0, 0)",2022-07-27T19:41:37,167613398260,7255.991266666667,8342038
1,M016,2022-07-28,SAL,"(0, 0, 0)",2022-07-28T13:19:08,172464739370,7466.006033333333,2005727
2,M015,2022-08-01,RSK,"(3, 10, 30)",2022-08-01T16:30:01,168066949820,7275.625533333333,3368714
3,M015,2022-08-03,RSK,"(3, 10, 30)",2022-08-03T12:45:02,167609776180,7255.834466666666,1157254
4,M016,2022-08-04,RSK,"(3, 10, 30)",2022-08-04T15:45:22,167712443360,7260.278933333333,1293164


In [39]:
path = session_files(sessions.loc[0].path)['hpc']['lfp'].replace('.bin', '.meta')
md = load_metadata(f'{root}/{path}')

In [41]:
# get the signal sampling rate in Hz
samplerate = float(md['imSampRate'])

In [42]:
samplerate

30000.0

In [43]:
# get a number that converts from raw 16-bit memory map values to volts (pre-gain correction)
int2volt = float(md['imAiRangeMax']) / int(md.get('imMaxInt', 512))

In [44]:
int2volt

0.001171875

In [47]:
params

acqApLfSy            384,384,1
gateMode             Immediate
imAiRangeMax               0.6
imAiRangeMin              -0.6
imCalibrated              true
imLEDEnable              false
imMaxInt                   512
imSampRate               30000
nSavedChans                385
snsApLfSy              384,0,1
snsSaveChanSubset    0:383,768
Name: 0, dtype: object

In [ ]:
''

In [40]:
ap, lf, sy = md['snsApLfSy'].split(',')
chancounts = {'AP': int(ap), 'LF': int(lf), 'SY': int(sy)}
assert chancounts['SY'] == 1
assert chancounts['LF'] == len(chanidx) - 1

probetype = int(md.get('imDatPrb_type', 0))
assert probetype == 0

gains = md['imroTbl'].strip('(').strip(')')
gains = gains.split(')(')[1:]
lfgain = np.array([int(s.split(' ')[4]) for s in gains])
assert np.all(lfgain == lfgain[0])
lfgain = lfgain[0]

nbytes = int(md['fileSizeBytes'])
nchans = int(md['nSavedChans'])
assert nbytes % 2 == 0
assert nbytes % nchans == 0
nsamples = nbytes // 2 // nchans

ValueError: not enough values to unpack (expected 2, got 1)